## Now for the Analysis
After lots of munging, I have a dataframe containing text of all the presidential inaugural addresses, along with metadata like date, wordcounts, party affilations, and etc.

First we open the dataframe

In [11]:
import pandas as pd
inaugurals = pd.read_csv('./data/inaugurals_complete.csv')
inaugurals.head()


,urls,years,pres,texts,wordcounts,party,change,new counts
0,http://avalon.law.yale.edu/18th_century/wash1.asp,1789,George Washington,"of THE CITY OF NEW YORK THURSDAY, APRIL 3...",1443.0,Federalist,New_Party,1443
1,http://avalon.law.yale.edu/18th_century/wash2.asp,1793,George Washington,"of THE CITY OF PHILADELPHIA MONDAY, MARCH...",149.0,Federalist,Incumbent_Party,149
2,http://avalon.law.yale.edu/18th_century/adams.asp,1797,John Adams,© 2008 Lillian Goldman Law Library 127 Wall St...,2417.0,Democratic-Republican,New_Party,2417
3,http://avalon.law.yale.edu/19th_century/jefina...,1801,Thomas Jefferson,"March 4, 1801 FRIENDS AND FELLOW-...",1742.0,Democratic-Republican,Incumbent_Party,1742
4,http://avalon.law.yale.edu/19th_century/jefina...,1805,Thomas Jefferson,"March 4, 1805 Proceeding, fellow citi...",2162.0,Democratic-Republican,Incumbent_Party,2162


In [5]:
import os, csv, math
import pandas as pd
import numpy as np

from collections import Counter
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer



## Now do some basic word-counting, using Ted's functions from Week 4

In [6]:
def tokenize(astring):
    ''' Breaks a string into words, and counts them.
    Designed so it strips punctuation and lowercases everything,
    but doesn't separate hashtags and at-signs.
    '''
    wordcounts = Counter()
    # create a counter to hold the counts
    
    tokens = astring.split()
    for t in tokens:
        word = t.strip(',.!?:;-—()<>[]/"\'').lower()
        wordcounts[word] += 1
        
    return wordcounts

def addcounters(counter2add, countersum):
    ''' Adds all the counts in counter2add to countersum.
    Because Counters(like dictionaries) are mutable, it
    doesn't need to return anything.
    '''
    
    for key, value in counter2add.items():
        countersum[key] += value

def create_vocab(seq_of_strings, n):
    ''' Given a sequence of text snippets, this function
    returns the n most common words. We'll use this to
    create a limited 'vocabulary'.
    '''
    vocab = Counter()
    for astring in seq_of_strings:
        counts = tokenize(astring)
        addcounters(counts, vocab)
    topn = [x[0] for x in vocab.most_common(n)]
    return topn

# Let's test the vocabulary function.
vocab = create_vocab(inaugurals['texts'], 4000)
vocab[0:10]
        

['the', 'of', 'and', 'to', 'in', 'a', 'our', 'that', 'we', 'be']

In [7]:
def logodds(countsA, countsB, word):
    ''' Straightforward.
    '''
    
    odds = (countsA[word] + 1) / (countsB[word] + 1)
    
    # Why do we add 1 on both sides? Two reasons. The hacky one is 
    # that otherwise we'll get a division-by-zero error whenever
    # word isn't present in countsB. The more principled reason
    # is that this technique (called Laplacian smoothing) tends
    # to reduce the dramatic disproportion likely to be found in
    # very rare words.
    
    return math.log(odds)

def signed_dunnings(countsA, totalA, countsB, totalB, word):
    ''' Less straightforward. This function calculates a signed (+1 / -1)
    version of Dunning's log likelihood. Intuitively, this is a number 
    that gets larger as the frequency of the word in our two corpora
    diverges from its EXPECTED frequency -- i.e., the frequency it would
    have if it were equally distributed over both. But it also tends to get
    larger as the raw frequency of the word increases.
    
    Note that this function requires two additional arguments:
    the total number of words in A and B. We could calculate that inside
    the function, but it's faster to calculate it just once, outside the function.
    
    Also note: the strict definition of Dunnings has no 'sign': it gets bigger
    whether a word is overrepresented in A or B. I've edited that so that Dunnings
    is positive if overrepresented in A, and negative if overrepresented in B.
    '''
    if word not in countsA and word not in countsB:
        return 0
    
    # the raw frequencies of this word in our two corpora
    # still doing a little Laplacian smoothing here
    a = countsA[word] + 0.1
    b = countsB[word] + 0.1
    
    # now let's calculate the expected number of times this
    # word would occur in both if the frequency were constant
    # across both
    overallfreq = (a + b) / (totalA + totalB)
    expectedA = totalA * overallfreq
    expectedB = totalB * overallfreq
    
    # and now the Dunning's formula
    dunning = 2 * ((a * math.log(a / expectedA)) + (b * math.log(b / expectedB)))
    
    if a < expectedA:
        return -dunning
    else:   
        return dunning

# a set of common words is often useful
stopwords = {'a', 'an', 'are', 'and', 'but', 'or', 'that', 'this', 'so', 
             'all', 'at', 'if', 'in', 'i', 'is', 'was', 'by', 'of', 'to', 
             'the', 'be', 'you', 'were'}

# finally, one more function: given a list of tuples like
testlist = [(10, 'ten'), (2000, 'two thousand'), (0, 'zero'), (-1, 'neg one'), (8, 'eight')]
# we're going to want to sort them and print the top n and bottom n

def headandtail(tuplelist, n):
    tuplelist.sort(reverse = True)
    print("TOP VALUES:")
    for i in range(n):
        print(tuplelist[i][1], tuplelist[i][0])
    
    print()
    print("BOTTOM VALUES:")
    lastindex = len(tuplelist) - 1
    for i in range(lastindex, lastindex - n, -1):
        print(tuplelist[i][1], tuplelist[i][0])
        
headandtail(testlist, 2)
    

TOP VALUES:
two thousand 2000
ten 10

BOTTOM VALUES:
neg one -1
zero 0


In [15]:
# Code for Exercise 1

# Start by creating a vocabulary for words in the Trump tweets.
# Put it in a variable called 'vocab'.

inaugural_text = inaugurals['texts']
vocab = create_vocab(inaugural_text, 5000)
vocab[:10]

# Remember the function create_vocab takes two arguments:
# (seq_of_strings, n)
# We can afford to include all the words, so set n for 5000.


# An optional step: removing stopwords
vocab = list(set(vocab) - stopwords)

# Create counters for the android and iphone corpora.

Democrat = Counter()
Republican = Counter()

# Figure out how many rows are in the Trump DataFrame
# and put that number in a variable like 'numrows.'
# Then iterate through the 'text' column of the data frame.

numrows = 58

# for each text cell, get a Counter with words counts for that cell
# then add those counts either to iphone or android, like so:

for i in range(numrows):
    counts = tokenize(inaugurals['texts'][i])
    if 'Democrat' in inaugurals['party'][i]:
        addcounters(counts, Democrat)
    elif 'Republican' in inaugurals['party'][i]:
        addcounters(counts, Republican)

        
# print(type(android))
# When you get around to running Dunning's, you'll need to
# create variables that hold the total count of *all words*
# in iphone and android.
total_democrat = sum(Democrat.values())
total_republican = sum(Republican.values())

# Create an empty list to hold pairs of (overrepresentation_measure, word)
# Then iterate through your vocabulary. For each word, measure 
# overrepresentation using either logodds or signed_dunnings.
# Create a tuple, (overrepresentation_measure, word)
# and append it to the empty list you created.

new_list = []
for word in vocab:
#     g = logodds(Republican, Democrat, word)
    g = signed_dunnings(Republican, total_republican, Democrat, total_democrat, word)
    new_list.append((g, word))
    

# Finally use the headandtail function to display the top 25 and bottom 25
# words in your tuplelist.
headandtail(new_list, 20)

TOP VALUES:
law 37.85900298929792
business 32.87730435677244
enforcement 27.661956717563825
there 26.71258693822815
no 21.023840498701418
we 20.368952478920193
freedom 19.949151468604846
will 19.711108571025164
make 19.27829134912094
do 18.29887958039273
negro 17.7213002579283
amendment 17.662006465968325
accept 17.01798088257794
congress 16.84090924743422
america 16.585253111580684
method 14.88972168638792
islands 14.88972168638792
arbitration 14.88972168638792
south 14.304357332097728
prayer 13.7751544999991

BOTTOM VALUES:
which -25.587694865809624
been -24.768251604369084
powerful -24.257396715885317
powers -20.639945618320855
union -19.808186401517098
myself -19.68520084932692
me -19.212377774373138
foreign -18.94148581831667
my -18.87101149359202
 -18.74433136968643
on -18.413088656786343
opinion -16.987689023401476
happy -16.890688937556916
fellow-citizens -16.81270986608169
spirit -16.686359964373743
period -16.515374263165533
limits -16.276798224583224
measures -15.84649594417